In [1]:
import base

import torch
import time
import numpy as np

from sigLoss import SigLoss, SigLoss_naive
from sigMMD import SigMMD, SigMMD_naive

import matplotlib.pyplot as plt

# Sig Loss gradients

In [2]:
rbf=True
sigma=1.

In [325]:
A = 1
M = 3
N = 2
D = 2

X = np.random.randn(A,M,D).cumsum(axis=1)
Y = np.random.randn(A,N,D).cumsum(axis=1)

X /= np.max(X)
Y /= np.max(Y)

In [326]:
X_naive = torch.tensor(X, dtype=torch.float64)
Y_naive = torch.tensor(Y, dtype=torch.float64)

X_cpu = X_naive.clone()
Y_cpu = Y_naive.clone()

# X_gpu = X_naive.clone().cuda()
# Y_gpu = Y_naive.clone().cuda()

X_naive.requires_grad = True
X_cpu.requires_grad = True
# X_gpu.requires_grad = True

n = 0
solver = 0

In [327]:
t = time.time()
l_naive = SigLoss_naive(n=n,solver=solver,rbf=rbf,sigma=sigma).forward(X_naive,Y_naive)
print('time:', np.round(time.time()-t,3), 's')
print(l_naive)

time: 0.007 s
tensor(4.8206, dtype=torch.float64, grad_fn=<AddBackward0>)


In [328]:
t = time.time()
l_cpu = SigLoss(n=n,solver=solver,rbf=rbf,sigma=sigma).forward(X_cpu,Y_cpu)
print('time:', np.round(time.time()-t,3), 's')
print(l_cpu)

time: 0.002 s
tensor(4.8206, dtype=torch.float64, grad_fn=<AddBackward0>)


In [329]:
# t = time.time()
# l_gpu = SigLoss(n=n,solver=solver).forward(X_gpu,Y_gpu)
# print('time:', np.round(time.time()-t,3), 's')
# print(l_gpu)

In [330]:
t = time.time()
l_naive.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.006 s


In [331]:
t = time.time()
l_cpu.backward()
print('time:', np.round(time.time()-t,3), 's')

time: 0.007 s


In [332]:
# t = time.time()
# l_gpu.backward()
# print('time:', np.round(time.time()-t,3), 's')

In [333]:
# torch.allclose(X_naive.grad, X_cpu.grad)

In [334]:
X_naive.grad

tensor([[[ 0.1033,  4.2963],
         [ 0.2582, -0.4314],
         [ 1.0158, -2.0655]]], dtype=torch.float64)

In [335]:
X_cpu.grad

tensor([[[ 0.0074,  3.0661],
         [-0.2805, -0.4208],
         [ 0.2539, -0.5164]]], dtype=torch.float64)

In [336]:
# torch.allclose(X_naive.grad, X_gpu.grad.cpu())

In [ ]:
# torch.allclose(X_cpu.grad, X_gpu.grad.cpu())

# Sig MMD gradients

In [ ]:
A = 1
B = 1
M = 3
N = 3
D = 2

X = np.random.randn(A,M,D).cumsum(axis=1)
Y = np.random.randn(B,N,D).cumsum(axis=1)

X /= np.max(X)
Y /= np.max(Y)

X_naive = torch.tensor(X, dtype=torch.float64)
Y_naive = torch.tensor(Y, dtype=torch.float64)

X_cpu = X_naive.clone()
Y_cpu = Y_naive.clone()

X_gpu = X_naive.clone().cuda()
Y_gpu = Y_naive.clone().cuda()

X_naive.requires_grad = True
X_cpu.requires_grad = True
X_gpu.requires_grad = True

In [ ]:
t = time.time()
mmd_naive = SigMMD_naive(n=n,solver=solver).forward(X_naive,Y_naive)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_naive)

In [ ]:
t = time.time()
mmd_cpu = SigMMD(n=n,solver=solver).forward(X_cpu,Y_cpu)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_cpu)

In [ ]:
t = time.time()
mmd_gpu = SigMMD(n=n,solver=solver).forward(X_gpu,Y_gpu)
print('time:', np.round(time.time()-t,3), 's')
print(mmd_gpu)

In [ ]:
t = time.time()
mmd_naive.backward()
print('time:', np.round(time.time()-t,3), 's')

In [ ]:
t = time.time()
mmd_cpu.backward()
print('time:', np.round(time.time()-t,3), 's')

In [ ]:
t = time.time()
mmd_gpu.backward()
print('time:', np.round(time.time()-t,3), 's')

In [ ]:
torch.allclose(X_naive.grad, X_cpu.grad)

In [ ]:
torch.allclose(X_naive.grad, X_gpu.grad.cpu())

In [ ]:
torch.allclose(X_cpu.grad, X_cpu.grad)